<a href="https://colab.research.google.com/github/AnwarMirza/SEP769-Cyber-Physical-Systems-McMaster-Summer2023/blob/main/04_RNN_Implementation_using_NumPy_CPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN Implementation using NumPy

- Sequence input, Char-level, Batch training, Python 3

- You can train RNNs using different vector representations. (Try "glove" or "word2vec" to train the sequence of words instead of the "One-hot-vector")

- This is a slightly modified version of the original code(Karpathy).

URL

-> https://github.com/JY-Yoon

-> Original code(Karpathy) : https://gist.github.com/karpathy/d4dee566867f8291f086



In [ ]:
import numpy as np

In [ ]:
# load text data

txt_data = "abcdefghijklmnopqrstuvwxyz abcdefghijklmnopqrstuvwxyz abcdefghijklmnopqrstuvwxyz " # input data
# txt_data = "I had neither kith nor kin in England, and was therefore as free as air."
# txt_data = open('input.txt', 'r').read() # test external files

chars = list(set(txt_data)) # split and remove duplicate characters. convert to list.

num_chars = len(chars) # the number of unique characters
txt_data_size = len(txt_data)

print("unique characters : ", num_chars) # You can see the number of unique characters in your input data.
print("txt_data_size : ", txt_data_size)

unique characters :  27
txt_data_size :  81


# One hot encoding

In [ ]:
# one hot encode
char_to_int = dict((c, i) for i, c in enumerate(chars)) # "enumerate" retruns index and value. Convert it to dictionary
int_to_char = dict((i, c) for i, c in enumerate(chars))
print(char_to_int)
print("----------------------------------------------------")
print(int_to_char)
print("----------------------------------------------------")
# integer encode input data
integer_encoded = [char_to_int[i] for i in txt_data] # "integer_encoded" is a list which has a sequence converted from an original data to integers.
print(integer_encoded)
print("----------------------------------------------------")
print("data length : ", len(integer_encoded))

{'t': 0, 'u': 1, 'h': 2, 'k': 3, 'g': 4, 'y': 5, 'q': 6, 'x': 7, 'j': 8, 'w': 9, 'b': 10, 'p': 11, 'd': 12, 'o': 13, 'z': 14, ' ': 15, 'f': 16, 'e': 17, 'n': 18, 'l': 19, 'c': 20, 'r': 21, 'a': 22, 's': 23, 'v': 24, 'm': 25, 'i': 26}
----------------------------------------------------
{0: 't', 1: 'u', 2: 'h', 3: 'k', 4: 'g', 5: 'y', 6: 'q', 7: 'x', 8: 'j', 9: 'w', 10: 'b', 11: 'p', 12: 'd', 13: 'o', 14: 'z', 15: ' ', 16: 'f', 17: 'e', 18: 'n', 19: 'l', 20: 'c', 21: 'r', 22: 'a', 23: 's', 24: 'v', 25: 'm', 26: 'i'}
----------------------------------------------------
[22, 10, 20, 12, 17, 16, 4, 2, 26, 8, 3, 19, 25, 18, 13, 11, 6, 21, 23, 0, 1, 24, 9, 7, 5, 14, 15, 22, 10, 20, 12, 17, 16, 4, 2, 26, 8, 3, 19, 25, 18, 13, 11, 6, 21, 23, 0, 1, 24, 9, 7, 5, 14, 15, 22, 10, 20, 12, 17, 16, 4, 2, 26, 8, 3, 19, 25, 18, 13, 11, 6, 21, 23, 0, 1, 24, 9, 7, 5, 14, 15]
----------------------------------------------------
data length :  81


In [ ]:
# Not actually used.

onehot_encoded = []

for ix in integer_encoded: # ix is an index mapped to a unique character.
    letter = [0 for _ in range(len(chars))] # A list len is equal to the number of unique characters and whose elements are all zero.
    letter[ix] = 1 # 'letter' is a one-hot vector.
    onehot_encoded.append(letter) # Add a 1d list(a vector for one character).
onehot_encoded = np.array(onehot_encoded) # list to np-array

print(onehot_encoded.shape)     #  = (len(data),len(chars))
print(onehot_encoded)

# invert encoding
inverted = int_to_char[np.argmax(onehot_encoded[0])] # "argmax" returns the index of the largest value.
print(inverted)


(81, 27)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
a


# Initialization

In [ ]:
# hyperparameters

iteration = 5000
sequence_length = 10
batch_size = round((txt_data_size /sequence_length)+0.5) # = math.ceil
hidden_size = 100  # size of hidden layer of neurons.
learning_rate = 1e-1


# model parameters

W_xh = np.random.randn(hidden_size, num_chars)*0.01     # weight input -> hidden.
W_hh = np.random.randn(hidden_size, hidden_size)*0.01   # weight hidden -> hidden
W_hy = np.random.randn(num_chars, hidden_size)*0.01     # weight hidden -> output

b_h = np.zeros((hidden_size, 1)) # hidden bias
b_y = np.zeros((num_chars, 1)) # output bias

h_prev = np.zeros((hidden_size,1)) # h_(t-1)

# Forward propagation

In [ ]:
def forwardprop(inputs, targets, h_prev):

    # Since the RNN receives the sequence, the weights are not updated during one sequence.
    xs, hs, ys, ps = {}, {}, {}, {} # dictionary
    hs[-1] = np.copy(h_prev) # Copy previous hidden state vector to -1 key value.
    loss = 0 # loss initialization

    for t in range(len(inputs)): # t is a "time step" and is used as a key(dic).

        xs[t] = np.zeros((num_chars,1))
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(W_xh, xs[t]) + np.dot(W_hh, hs[t-1]) + b_h) # hidden state.
        ys[t] = np.dot(W_hy, hs[t]) + b_y # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars.
        # Softmax. -> The sum of probabilities is 1 even without the exp() function, but all of the elements are positive through the exp() function.

        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss). Efficient and simple code

#         y_class = np.zeros((num_chars, 1))
#         y_class[targets[t]] =1
#         loss += np.sum(y_class*(-np.log(ps[t]))) # softmax (cross-entropy loss)

    return loss, ps, hs, xs

# Backward propagation

In [ ]:
def backprop(ps, inputs, hs, xs):

    dWxh, dWhh, dWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy) # make all zero matrices.
    dbh, dby = np.zeros_like(b_h), np.zeros_like(b_y)
    dhnext = np.zeros_like(hs[0]) # (hidden_size,1)

    # reversed
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t]) # shape (num_chars,1).  "dy" means "dloss/dy"
        dy[targets[t]] -= 1 # backprop into y. After taking the soft max in the input vector, subtract 1 from the value of the element corresponding to the correct label.
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(W_hy.T, dy) + dhnext # backprop into h.
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity #tanh'(x) = 1-tanh^2(x)
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(W_hh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients.

    return dWxh, dWhh, dWhy, dbh, dby

# Training

In [ ]:
data_pointer = 0

# memory variables for Adagrad
mWxh, mWhh, mWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
mbh, mby = np.zeros_like(b_h), np.zeros_like(b_y)


for i in range(iteration):
    h_prev = np.zeros((hidden_size,1)) # reset RNN memory
    data_pointer = 0 # go from start of data

    for b in range(batch_size):

        inputs = [char_to_int[ch] for ch in txt_data[data_pointer:data_pointer+sequence_length]]
        targets = [char_to_int[ch] for ch in txt_data[data_pointer+1:data_pointer+sequence_length+1]] # t+1

        if (data_pointer+sequence_length+1 >= len(txt_data) and b == batch_size-1): # processing of the last part of the input data.
#             targets.append(char_to_int[txt_data[0]])   # When the data doesn't fit, add the first char to the back.
            targets.append(char_to_int[" "])   # When the data doesn't fit, add space(" ") to the back.


        # forward
        loss, ps, hs, xs = forwardprop(inputs, targets, h_prev)
#         print(loss)

        # backward
        dWxh, dWhh, dWhy, dbh, dby = backprop(ps, inputs, hs, xs)


    # perform parameter update with Adagrad
        for param, dparam, mem in zip([W_xh, W_hh, W_hy, b_h, b_y],
                                    [dWxh, dWhh, dWhy, dbh, dby],
                                    [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam # elementwise
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

        data_pointer += sequence_length # move data pointer

    if i % 100 == 0:
        print ('iter %d, loss: %f' % (i, loss)) # print progress



iter 0, loss: 2.795307
iter 100, loss: 0.014536
iter 200, loss: 0.006334
iter 300, loss: 0.003968
iter 400, loss: 0.002879
iter 500, loss: 0.002255
iter 600, loss: 0.001845
iter 700, loss: 0.001556
iter 800, loss: 0.001341
iter 900, loss: 0.001176
iter 1000, loss: 0.001047
iter 1100, loss: 0.000942
iter 1200, loss: 0.000856
iter 1300, loss: 0.000784
iter 1400, loss: 0.000723
iter 1500, loss: 0.000670
iter 1600, loss: 0.000625
iter 1700, loss: 0.000585
iter 1800, loss: 0.000549
iter 1900, loss: 0.000518
iter 2000, loss: 0.000490
iter 2100, loss: 0.000465
iter 2200, loss: 0.000442
iter 2300, loss: 0.000421
iter 2400, loss: 0.000402
iter 2500, loss: 0.000385
iter 2600, loss: 0.000369
iter 2700, loss: 0.000354
iter 2800, loss: 0.000340
iter 2900, loss: 0.000328
iter 3000, loss: 0.000316
iter 3100, loss: 0.000305
iter 3200, loss: 0.000295
iter 3300, loss: 0.000285
iter 3400, loss: 0.000276
iter 3500, loss: 0.000267
iter 3600, loss: 0.000259
iter 3700, loss: 0.000252
iter 3800, loss: 0.00024

# Prediction

In [ ]:
def predict(test_char, length):
    x = np.zeros((num_chars, 1))
    x[char_to_int[test_char]] = 1
    ixes = []
    h = np.zeros((hidden_size,1))

    for t in range(length):
        h = np.tanh(np.dot(W_xh, x) + np.dot(W_hh, h) + b_h)
        y = np.dot(W_hy, h) + b_y
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(num_chars), p=p.ravel()) # ravel -> rank0
        # "ix" is a list of indexes selected according to the soft max probability.
        x = np.zeros((num_chars, 1)) # init
        x[ix] = 1
        ixes.append(ix) # list
    txt = ''.join(int_to_char[i] for i in ixes)
    print ('----\n %s \n----' % (txt, ))

In [ ]:
predict('a',30) # (char, len of output)

----
 bcdefghijklmnopqrstuvwxyz abcd 
----


In [ ]:
predict('b',30)

----
 cdefghijklmnopqrstuvwxyz abcde 
----


In [ ]:
predict('c',30)

----
 defghijklmnopqrstuvwxyz abcdef 
----
